Import Libraries

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt 
from matplotlib import cm
import os
import math
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, Callback
import tensorflow.keras.backend as K

from IPython.display import clear_output
from time import time, gmtime, strftime, localtime
import csv
import sys
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from scipy import pi, exp

import seaborn as sns

## run on CPUs...
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# disable eager excution
tf.compat.v1.disable_eager_execution()

In [ ]:
## Define Geometry and Flow parameters
## Here we define main parameters for running the program
## pinn_grid = True will be the classic Raissi's grid and method with n = 51
## with pinn_grid = False, you can choose your own grid, simulation solution
## (set to random)
## Re is Reynolds number
## EPOCH is number of iterations. At least 10K iterations are required for
## reasonable solution

## Nsamples is sample size, Originally was set to 405. It should not be lower
## than this. It can be higher for better solution.

## Nbatch, number of batches. Multiple of 32 is a good number.

## Restart_sol = False is the intitial option. With this option set to True
## you can restart a soution from last stage. This is very important option
## as it helps to restart from unexpected power or inernet failue.

## x_l, x_u, y_l, y_u are geometric limits of computational domain. You can
## set y_l to -1.5 or -4 to have a deep cavity grid.

## vdx and vdy are grid difference in x and y directions

## When running a notebook, make sure to restart the kernel for least overhead.

## checkpoint_path is an important directory. Its name must be changed to 
## reflect run conditions. Failure to do will overwrite good saved soutions.
## You will experience its importance as you run many cases. I have 
## incorprated some input dialogues for safe starts.

## checkpoint_dir is the directory name.

## For boundary conditions, look in create_nn function. Make necessary 
## changes in required bc_top, bc_bottom, bc_left and bc_right statements.

## Happy computing. Always Discover something new and correct deficiencies
## in the code for the powering future knowledge and skills. Thank you.

pinn_grid = False
Re = 400.
EPOCH = 2500
Nsamples = 405
Nbatch = 32
n = 41
Restart_sol = False

x_l, x_u, y_l, y_u = 0, 1, -1, 0
vdx = (x_u - x_l) / (n - 1) 
vdy = (y_u - y_l) / (n - 1)

resp_op = input('Are you starting a new solution, y for yes, n for no :')
if (resp_op == 'y'):
   Restart_sol = False
   checkpoint_path = input('Prescribe a meaningful path name like Re400_tr1/cp.ckpt etc :')
else:
   Restart_sol = True 
   print('Make sure you use correct checkpoint_path')
   checkpoint_path = input('Input previously saved path :')

checkpoint_dir = os.path.dirname(checkpoint_path)

### 1. PINN for 2D N-S equations (lid driven cavity)

    1. Incompressible Navier–Stokes equation for 2D fluid case: u(x, y), v(x, y), p(x, y)    , x∈[0, 1], y∈[-1, 0]
        - Continuity equation : u_x + v_y = 0
        - Momentum equation 1 : u*u_x + v*u_y = -p_x + vis*(u_xx + u_yy)
        - Momentum equation 2 : u*v_x + v*v_y = -p_y + vis*(v_xx + v_yy)
        
    2. BC:
        - Top    : u = 1 , v = 0 , p_n = 0
        - Left   : u = 0 , v = 0 , p_n = 0
        - Right  : u = 0 , v = 0 , p_n = 0
        - Bottom : u = 0 , v = 0 , p_n = 0
         
    3. Constants, coefficients:
        - density = 1, therefore not appear in momentum equations
        - vis: kinematic viscosity (1 / Re)

#### 1.0. Data

In [ ]:
# define sampling plan
class SamplingPlan_Fix(keras.utils.Sequence):
    
    def __init__(self, data=( ), batch_size=( ), batch_per_epoch=1):
        # sampling plan: data=(X_pde, y_pde, X_ic, y_ic, X_bc, y_bc), batch_size=(n_pde, n_ic, n_bc)
        self.X, self.y, self.X_ic, self.y_ic, self.X_bc, self.y_bc = data
        self.n, self.n_ic, self.n_bc = len(self.X), len(self.X_ic), len(self.X_bc)
        self.ID, self.ID_ic, self.ID_bc = np.arange(self.n), np.arange(self.n_ic), np.arange(self.n_bc)
        # input parameters
        self.batch_size, self.batch_ic, self.batch_bc = batch_size
        self.batch_per_epoch = batch_per_epoch
        
    def __len__(self):
        # number of mini batch per epoch
        return self.batch_per_epoch
    
    def __getitem__(self, idx):
        # shuffle & pick collocation sample
        np.random.shuffle(self.ID)
        idxs = self.ID[:self.batch_size]
        batch_X, batch_y = self.X[idxs], self.y[idxs]
        # shuffle & pick ic sample
        np.random.shuffle(self.ID_ic)
        idxs = self.ID_ic[:self.batch_ic]
        batch_X_ic, batch_y_ic = self.X_ic[idxs], self.y_ic[idxs]            
        # shuffle & pick bc sample 
        np.random.shuffle(self.ID_bc)
        idxs = self.ID_bc[:self.batch_bc]
        batch_X_bc, batch_y_bc = self.X_bc[idxs], self.y_bc[idxs]
        # combine all sample
        batch_X, batch_y = np.vstack([batch_X, batch_X_ic, batch_X_bc]), np.vstack([batch_y, batch_y_ic, batch_y_bc])        
        return (batch_X, batch_y)

#### 1.3. a-PINN / n-PINN / can-PINN

In [ ]:
# specify a-PINN / n-PINN / can-PINN
def create_nn(scheme, ff, n_ffs, sigma, lmbda, n_nodes, acf, lr_int):
    # input layers -> split into (x, y, dx, dy)
    inputs = layers.Input(shape=(4,))
    x, y, dx, dy = layers.Lambda( lambda k: tf.split(k, num_or_size_splits=4, axis=1))(inputs)
    
    # features mapping
    initializer_ff = tf.keras.initializers.TruncatedNormal(stddev=sigma)  # features initializer
    
    if (ff == 'FF'):
        hidden_f0 = layers.Dense(n_ffs, activation='linear', use_bias=False, kernel_initializer=initializer_ff)(layers.Concatenate()([x, y]))
        hidden_sin, hidden_cos = tf.math.sin(2*tf.constant(pi)*hidden_f0), tf.math.cos(2*tf.constant(pi)*hidden_f0)
        hidden_ff = layers.Concatenate()([hidden_sin, hidden_cos])
        
    if (ff == 'SF') or (ff == 'SIREN'):
        hidden_f0 = layers.Dense(n_ffs*2, activation='linear', kernel_initializer=initializer_ff)(layers.Concatenate()([x, y]))
        hidden_ff = tf.math.sin(2*tf.constant(pi)*hidden_f0)

    if (ff == 'HF'):
        hidden_ff = layers.Dense(n_ffs*2, activation=acf, kernel_initializer=initializer_ff)(layers.Concatenate()([x, y]))

    # hidden layers
    if (ff == 'SIREN'):
        initializer = tf.keras.initializers.HeUniform()  # hidden layers initializer
        hidden_1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_ff)
        hidden_2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_1)
        hidden_l = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_2)
    else:
        initializer = tf.keras.initializers.GlorotUniform()  # hidden layers initializer
        hidden_1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_ff)
        hidden_2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_1)
        hidden_l = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_2)

    # split layers - u
    if (ff == 'SIREN'):
        hidden_u1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_l)
        hidden_u2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_u1)
        hidden_ul = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_u2)
    else:
        hidden_u1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_l)
        hidden_u2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_u1)
        hidden_ul = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_u2)

    # split layers - v
    if (ff == 'SIREN'):
        hidden_v1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_l)
        hidden_v2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_v1)
        hidden_vl = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_v2)
    else:
        hidden_v1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_l)
        hidden_v2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_v1)
        hidden_vl = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_v2)  
        
    # split layers - p
    if (ff == 'SIREN'):
        hidden_p1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_l)
        hidden_p2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_p1)
        hidden_pl = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_p2)
    else:
        hidden_p1 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_l)
        hidden_p2 = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_p1)
        hidden_pl = layers.Dense(n_nodes, activation=acf, kernel_initializer=initializer)(hidden_p2)          
        
    # output layers
    u = layers.Dense(1, use_bias=False, name="U")(hidden_ul)
    v = layers.Dense(1, use_bias=False, name="V")(hidden_vl)
    p = layers.Dense(1, use_bias=False, name="P")(hidden_pl)  
    
    # initiate model
    outputs = layers.Concatenate()([u, v, p]) 
    nn = models.Model(inputs=inputs, outputs=outputs)
    
    # axillary PDE outputs
    u_x, u_y = K.gradients(u, x)[0], K.gradients(u, y)[0]
    v_x, v_y = K.gradients(v, x)[0], K.gradients(v, y)[0]
    p_x, p_y = K.gradients(p, x)[0], K.gradients(p, y)[0]
    u_xx, u_yy = K.gradients(u_x, x)[0], K.gradients(u_y, y)[0]
    v_xx, v_yy = K.gradients(v_x, x)[0], K.gradients(v_y, y)[0]    

    # initial & boundary conditions:
    # Top    : u = 1 , v = 0
    # Left   : u = 0 , v = 0
    # Right  : u = 0 , v = 0
    # Bottom : u = 0 , v = 0
    _top, _bottom = tf.equal(y, y_u), tf.equal(y, y_l)
    _left, _right = tf.equal(x, x_l), tf.equal(x, x_u)
    _bc = tf.logical_or( tf.logical_or(_top, _bottom) , tf.logical_or(_left, _right) )

    u_top, v_top = tf.boolean_mask(u, _top), tf.boolean_mask(v, _top)
    bc_top = tf.compat.v1.losses.mean_squared_error(labels=tf.ones_like(u_top), predictions=u_top) + \
             tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(v_top), predictions=v_top)

    u_left, v_left = tf.boolean_mask(u, _left & ~_top), tf.boolean_mask(v, _left & ~_top)
    bc_left = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(u_left), predictions=u_left) + \
              tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(v_left), predictions=v_left)

    u_right, v_right = tf.boolean_mask(u, _right & ~_top), tf.boolean_mask(v, _right & ~_top)
    bc_right = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(u_right), predictions=u_right) + \
               tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(v_right), predictions=v_right)

    u_bottom, v_bottom = tf.boolean_mask(u, _bottom), tf.boolean_mask(v, _bottom)
    bc_bottom = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(u_bottom), predictions=u_bottom) + \
                tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(v_bottom), predictions=v_bottom)
    
    bc_mse = bc_top + bc_left + bc_right + bc_bottom
    
    # PDE (NS equation)
    # Continuity equation : u_x + v_y = 0
    # Momentum equation 1 : u_t + u*u_x + v*u_y = -(1/rho)*p_x + nu*(u_xx + u_yy)
    # Momentum equation 2 : v_t + u*v_x + v*v_y = -(1/rho)*p_y + nu*(v_xx + v_yy)
    
    # auto-differentian PDE (a-pde)
    a_residuals_continuity = u_x + v_y
    a_residuals_momentum_1 = u*u_x + v*u_y + p_x - 1.0/Re*(u_xx + u_yy)
    a_residuals_momentum_2 = u*v_x + v*v_y + p_y - 1.0/Re*(v_xx + v_yy)
    
    # exclude BC points 
    a_residuals_continuity = tf.boolean_mask(a_residuals_continuity, ~_bc)
    a_residuals_momentum_1 = tf.boolean_mask(a_residuals_momentum_1, ~_bc)
    a_residuals_momentum_2 = tf.boolean_mask(a_residuals_momentum_2, ~_bc)
    
    a_mse_continuity = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(a_residuals_continuity),
                                                              predictions=a_residuals_continuity)
    a_mse_momentum_1 = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(a_residuals_momentum_1),
                                                              predictions=a_residuals_momentum_1)
    a_mse_momentum_2 = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(a_residuals_momentum_2),
                                                              predictions=a_residuals_momentum_2)
    a_pde_mse = a_mse_continuity + a_mse_momentum_1 + a_mse_momentum_2
    a_pde_mse = a_pde_mse / lmbda     
    
    
    # numerical differentiation PDE (n-pde)
    # dx & dy get from input
    xE, xW = x + dx, x - dx
    yN, yS = y + dy, y - dy
    uvpE  = nn(tf.stack([xE, y, dx, dy], 1))
    uvpW  = nn(tf.stack([xW, y, dx, dy], 1))
    uvpN  = nn(tf.stack([x, yN, dx, dy], 1))
    uvpS  = nn(tf.stack([x, yS, dx, dy], 1))
    uE, vE, pE  = tf.split(uvpE, num_or_size_splits=3, axis=1)
    uW, vW, pW  = tf.split(uvpW, num_or_size_splits=3, axis=1)
    uN, vN, pN  = tf.split(uvpN, num_or_size_splits=3, axis=1)
    uS, vS, pS  = tf.split(uvpS, num_or_size_splits=3, axis=1)
    
    # second order
    xEE, xWW = x + 2.0*dx, x - 2.0*dx
    yNN, ySS = y + 2.0*dy, y - 2.0*dy    
    uvpEE = nn(tf.stack([xEE, y, dx, dy], 1))
    uvpWW = nn(tf.stack([xWW, y, dx, dy], 1))
    uvpNN = nn(tf.stack([x, yNN, dx, dy], 1))
    uvpSS = nn(tf.stack([x, ySS, dx, dy], 1))  
    uEE, vEE, _ = tf.split(uvpEE, num_or_size_splits=3, axis=1)
    uWW, vWW, _ = tf.split(uvpWW, num_or_size_splits=3, axis=1)
    uNN, vNN, _ = tf.split(uvpNN, num_or_size_splits=3, axis=1)
    uSS, vSS, _ = tf.split(uvpSS, num_or_size_splits=3, axis=1)
    
    uc_e, uc_w = 0.5*(uE + u), 0.5*(uW + u) 
    vc_n, vc_s = 0.5*(vN + v), 0.5*(vS + v)
    div = (uc_e - uc_w) /dx + (vc_n - vc_s) /dy
    
    # 2nd upwind
    Uem_uw2 = 1.5*u  - 0.5*uW
    Uep_uw2 = 1.5*uE - 0.5*uEE  
    Uwm_uw2 = 1.5*uW - 0.5*uWW
    Uwp_uw2 = 1.5*u  - 0.5*uE
    Ue_uw2 = tf.where(tf.greater_equal(uc_e, 0.0), Uem_uw2, Uep_uw2)
    Uw_uw2 = tf.where(tf.greater_equal(uc_w, 0.0), Uwm_uw2, Uwp_uw2)
        
    Unm_uw2 = 1.5*u  - 0.5*uS
    Unp_uw2 = 1.5*uN - 0.5*uNN    
    Usm_uw2 = 1.5*uS - 0.5*uSS
    Usp_uw2 = 1.5*u  - 0.5*uN
    Un_uw2 = tf.where(tf.greater_equal(vc_n, 0.0), Unm_uw2, Unp_uw2)
    Us_uw2 = tf.where(tf.greater_equal(vc_s, 0.0), Usm_uw2, Usp_uw2)

    Vem_uw2 = 1.5*v  - 0.5*vW
    Vep_uw2 = 1.5*vE - 0.5*vEE
    Vwm_uw2 = 1.5*vW - 0.5*vWW
    Vwp_uw2 = 1.5*v  - 0.5*vE
    Ve_uw2 = tf.where(tf.greater_equal(uc_e, 0.0), Vem_uw2, Vep_uw2)
    Vw_uw2 = tf.where(tf.greater_equal(uc_w, 0.0), Vwm_uw2, Vwp_uw2)
        
    Vnm_uw2 = 1.5*v  - 0.5*vS
    Vnp_uw2 = 1.5*vN - 0.5*vNN    
    Vsm_uw2 = 1.5*vS - 0.5*vSS
    Vsp_uw2 = 1.5*v  - 0.5*vN
    Vn_uw2 = tf.where(tf.greater_equal(vc_n, 0.0), Vnm_uw2, Vnp_uw2)
    Vs_uw2 = tf.where(tf.greater_equal(vc_s, 0.0), Vsm_uw2, Vsp_uw2)
        
    UUx_uw2 = (uc_e*Ue_uw2 - uc_w*Uw_uw2) /dx
    VUy_uw2 = (vc_n*Un_uw2 - vc_s*Us_uw2) /dy
    UVx_uw2 = (uc_e*Ve_uw2 - uc_w*Vw_uw2) /dx
    VVy_uw2 = (vc_n*Vn_uw2 - vc_s*Vs_uw2) /dy
    
    # 2nd central difference    
    Uxx_cd2 = (uE - 2.0*u + uW)/ (dx*dx) 
    Uyy_cd2 = (uN - 2.0*u + uS)/ (dy*dy) 
    Vxx_cd2 = (vE - 2.0*v + vW)/ (dx*dx) 
    Vyy_cd2 = (vN - 2.0*v + vS)/ (dy*dy) 

    pe_cd2 = (p + pE) /2.0 
    pw_cd2 = (pW + p) /2.0 
    pn_cd2 = (p + pN) /2.0 
    ps_cd2 = (pS + p) /2.0 
    
    Px_cd2 = (pe_cd2 - pw_cd2) /dx
    Py_cd2 = (pn_cd2 - ps_cd2) /dy
        
    n_residuals_continuity = div
    n_residuals_momentum_1 = UUx_uw2 + VUy_uw2 - 1.0/Re *(Uxx_cd2 + Uyy_cd2) - u*div + Px_cd2
    n_residuals_momentum_2 = UVx_uw2 + VVy_uw2 - 1.0/Re *(Vxx_cd2 + Vyy_cd2) - v*div + Py_cd2   
    
    # exclude BC points 
    n_residuals_continuity = tf.boolean_mask(n_residuals_continuity, ~_bc)
    n_residuals_momentum_1 = tf.boolean_mask(n_residuals_momentum_1, ~_bc)
    n_residuals_momentum_2 = tf.boolean_mask(n_residuals_momentum_2, ~_bc)
    
    n_mse_continuity = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(n_residuals_continuity),
                                                              predictions=n_residuals_continuity)
    n_mse_momentum_1 = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(n_residuals_momentum_1),
                                                              predictions=n_residuals_momentum_1)
    n_mse_momentum_2 = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(n_residuals_momentum_2),
                                                              predictions=n_residuals_momentum_2)
    n_pde_mse = n_mse_continuity + n_mse_momentum_1 + n_mse_momentum_2
    n_pde_mse = n_pde_mse / lmbda     
    
    
    # coupled automatic-numerical differentiation PDE (can-pde)
    uE_x, uW_x = K.gradients(uE, xE)[0], K.gradients(uW, xW)[0]
    uN_y, uS_y = K.gradients(uN, yN)[0], K.gradients(uS, yS)[0]
    
    vE_x, vW_x = K.gradients(vE, xE)[0], K.gradients(vW, xW)[0]
    vN_y, vS_y = K.gradients(vN, yN)[0], K.gradients(vS, yS)[0]   
    
    pE_x, pW_x = K.gradients(pE, xE)[0], K.gradients(pW, xW)[0]
    pN_y, pS_y = K.gradients(pN, yN)[0], K.gradients(pS, yS)[0]        
    
    # can 2nd upwind
    Uem_cuw2 = u  +  u_x*dx /2.0 #+ (uE_x - u_x)*dx /8.0
    Uep_cuw2 = uE - uE_x*dx /2.0 #+ (uE_x - u_x)*dx /8.0  
    Uwm_cuw2 = uW + uW_x*dx /2.0 #+ (u_x - uW_x)*dx /8.0
    Uwp_cuw2 = u  -  u_x*dx /2.0 #+ (u_x - uW_x)*dx /8.0
    Ue_cuw2 = tf.where(tf.greater_equal(uc_e, 0.0), Uem_cuw2, Uep_cuw2)
    Uw_cuw2 = tf.where(tf.greater_equal(uc_w, 0.0), Uwm_cuw2, Uwp_cuw2)    
    
    Unm_cuw2 = u  +  u_y*dy /2.0 #+ (uN_y - u_y)*dy /8.0
    Unp_cuw2 = uN - uN_y*dy /2.0 #+ (uN_y - u_y)*dy /8.0 
    Usm_cuw2 = uS + uS_y*dy /2.0 #+ (u_y - uS_y)*dy /8.0
    Usp_cuw2 = u  -  u_y*dy /2.0 #+ (u_y - uS_y)*dy /8.0
    Un_cuw2 = tf.where(tf.greater_equal(vc_n, 0.0), Unm_cuw2, Unp_cuw2)
    Us_cuw2 = tf.where(tf.greater_equal(vc_s, 0.0), Usm_cuw2, Usp_cuw2)

    Vem_cuw2 = v  +  v_x*dx /2.0 #+ (vE_x - v_x)*dx /8.0
    Vep_cuw2 = vE - vE_x*dx /2.0 #+ (vE_x - v_x)*dx /8.0
    Vwm_cuw2 = vW + vW_x*dx /2.0 #+ (v_x - vW_x)*dx /8.0
    Vwp_cuw2 = v  -  v_x*dx /2.0 #+ (v_x - vW_x)*dx /8.0
    Ve_cuw2 = tf.where(tf.greater_equal(uc_e, 0.0), Vem_cuw2, Vep_cuw2)
    Vw_cuw2 = tf.where(tf.greater_equal(uc_w, 0.0), Vwm_cuw2, Vwp_cuw2)
        
    Vnm_cuw2 = v  +  v_y*dy /2.0 #+ (vN_y - v_y)*dy /8.0
    Vnp_cuw2 = vN - vN_y*dy /2.0 #+ (vN_y - v_y)*dy /8.0
    Vsm_cuw2 = vS + vS_y*dy /2.0 #+ (v_y - vS_y)*dy /8.0
    Vsp_cuw2 = v  -  v_y*dy /2.0 #+ (v_y - vS_y)*dy /8.0
    Vn_cuw2 = tf.where(tf.greater_equal(vc_n, 0.0), Vnm_cuw2, Vnp_cuw2)
    Vs_cuw2 = tf.where(tf.greater_equal(vc_s, 0.0), Vsm_cuw2, Vsp_cuw2)    
    
    UUx_cuw2 = (uc_e*Ue_cuw2 - uc_w*Uw_cuw2) /dx
    VUy_cuw2 = (vc_n*Un_cuw2 - vc_s*Us_cuw2) /dy
    UVx_cuw2 = (uc_e*Ve_cuw2 - uc_w*Vw_cuw2) /dx
    VVy_cuw2 = (vc_n*Vn_cuw2 - vc_s*Vs_cuw2) /dy       
    
    # can 2nd central difference    
    pe_ccd2 = (p + pE) /2.0 - (pE_x - p_x)*dx /8.0
    pw_ccd2 = (pW + p) /2.0 - (p_x - pW_x)*dx /8.0
    pn_ccd2 = (p + pN) /2.0 - (pN_y - p_y)*dy /8.0
    ps_ccd2 = (pS + p) /2.0 - (p_y - pS_y)*dy /8.0
        
    Px_ccd2 = (pe_ccd2 - pw_ccd2) /dx
    Py_ccd2 = (pn_ccd2 - ps_ccd2) /dy    
    
    can_residuals_continuity = div
    can_residuals_momentum_1 = UUx_cuw2 + VUy_cuw2 - 1.0/Re *(Uxx_cd2 + Uyy_cd2) - u*div + Px_ccd2
    can_residuals_momentum_2 = UVx_cuw2 + VVy_cuw2 - 1.0/Re *(Vxx_cd2 + Vyy_cd2) - v*div + Py_ccd2

    # exclude BC points 
    can_residuals_continuity = tf.boolean_mask(can_residuals_continuity, ~_bc)
    can_residuals_momentum_1 = tf.boolean_mask(can_residuals_momentum_1, ~_bc)
    can_residuals_momentum_2 = tf.boolean_mask(can_residuals_momentum_2, ~_bc)
    
    can_mse_continuity = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(can_residuals_continuity),
                                                                predictions=can_residuals_continuity)
    can_mse_momentum_1 = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(can_residuals_momentum_1),
                                                                predictions=can_residuals_momentum_1)
    can_mse_momentum_2 = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(can_residuals_momentum_2),
                                                                predictions=can_residuals_momentum_2)
    can_pde_mse = can_mse_continuity + can_mse_momentum_1 + can_mse_momentum_2
    can_pde_mse = can_pde_mse / lmbda  
    
    
    # which method to use for PDE loss computation? a-PDE or n-PDE or can-PDE
    if (scheme == 'a-pde'):
        pde_mse = a_pde_mse
    if (scheme == 'n-pde'):
        pde_mse = n_pde_mse
    if (scheme == 'can-pde'):
        pde_mse = can_pde_mse    

        
    # optimizer
    optimizer = tf.keras.optimizers.Adam(lr_int)

    # compile model with [?] loss
    nn.compile(loss = compute_physics_loss(pde_mse, bc_mse),
               optimizer = optimizer,
               metrics = [compute_u_loss(dx), compute_v_loss(dy),
                          compute_bc_loss(bc_mse), compute_pde_loss(pde_mse)])

    # pathway to NN inside variables
    insiders = [u, v, p, pde_mse, bc_mse]
    eval_ins = K.function([nn.input, K.learning_phase()], insiders)   # evaluation function

    return (nn, eval_ins)


# loss functions
# define loss function (data loss)
def compute_data_loss():
    def data_loss(y_true, y_pred):
        return tf.compat.v1.losses.mean_squared_error(labels=y_true, predictions=y_pred)
    return data_loss

# define loss function (u loss)
def compute_u_loss(dx):
    def u_loss(y_true, y_pred):
        labels = tf.concat([tf.tile(tf.greater_equal(dx, 0), (1, 1)),
                            tf.tile(tf.equal(dx, -1), (1, 2))], 1)
        return tf.compat.v1.losses.mean_squared_error(labels=tf.boolean_mask(y_true, labels),
                                                      predictions=tf.boolean_mask(y_pred, labels))        
    return u_loss # return a function

# define loss function (v loss)
def compute_v_loss(dx):
    def v_loss(y_true, y_pred):
        labels = tf.concat([tf.tile(tf.equal(dx, -1), (1, 1)),
                            tf.tile(tf.greater_equal(dx, 0), (1, 1)),
                            tf.tile(tf.equal(dx, -1), (1, 1))], 1)
        return tf.compat.v1.losses.mean_squared_error(labels=tf.boolean_mask(y_true, labels),
                                                      predictions=tf.boolean_mask(y_pred, labels))    
    return v_loss # return a function

# define loss function (p loss)
def compute_p_loss(dx):
    def p_loss(y_true, y_pred):
        labels = tf.concat([tf.tile(tf.equal(dx, -1), (1, 2)),
                            tf.tile(tf.greater_equal(dx, 0), (1, 1))], 1)
        return tf.compat.v1.losses.mean_squared_error(labels=tf.boolean_mask(y_true, labels),
                                                      predictions=tf.boolean_mask(y_pred, labels))    
    return p_loss # return a function

# define loss function (physics loss)
def compute_physics_loss(pde_mse, bc_mse):
    def physics_loss(y_true, y_pred): return pde_mse + bc_mse
    return physics_loss # return a function

# define loss function (BC loss)
def compute_bc_loss(bc_mse):
    def bc_loss(y_true, y_pred): return bc_mse
    return bc_loss # return a function

# define loss function (PDE loss)
def compute_pde_loss(pde_mse):
    def pde_loss(y_true, y_pred): return pde_mse
    return pde_loss # return a function



# callback: training (prediction & residual) history
class TrainingHistory(Callback):
    def __init__(self, eval_ins, data):
        super(Callback, self).__init__()
        self.data = data
        self.eval_ins = eval_ins
        self.e_hist = []
        self.u_hist = []
        self.v_hist = []
        self.p_hist = []
        self.pde_mse_hist = []
        self.bc_mse_hist = []
    def on_epoch_end(self, epoch, logs={}):
        e = epoch + 1
        if (e < 10) | ((e < 100) & (e%10 == 0)) | ((e < 1000) & (e%100 == 0)) | (e%1000 == 0):
            data = self.data
            u, v, p, pde_mse, bc_mse = self.eval_ins(data)
            self.e_hist.append(e)
            self.u_hist.append(u)
            self.v_hist.append(v)
            self.p_hist.append(p)
            self.pde_mse_hist.append(pde_mse)
            self.bc_mse_hist.append(bc_mse)
            

### 2. Optimize D-PDE-NN: [SGD]

In [ ]:
#### 1.1. Geometry & BC
if (pinn_grid == False):
    nx = n
    ny = n
    # computational boundary
    ext = [x_l, x_u, y_l, y_u]
    x = np.linspace(x_l, x_u, nx)
    y = np.linspace(y_l, y_u, ny)

    # write a stacked csv file for x and y and read it for X_train
    
    header = ['x','y']
    with open('temp1.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for j in range(ny):
          for i in range (nx):
               data = [(x[i], y[j])]
        
               writer.writerows(data)
    f.close()

    #datadir = os.path.join(os.getcwd(), "../d00_data")
    sim = pd.read_csv( 'temp1.csv')
    sim['x'], sim['y'] = sim['x'], sim['y'] - y_u
    X_train = np.vstack([sim.x.values, sim.y.values, vdx * np.ones_like(sim.x.values), vdy * np.ones_like(sim.y.values)]).T
    y_train = np.random.rand(nx*ny,3)
else:
    # collect all .csv files from data folder
    datadir = os.path.join(os.getcwd(), "./d00_data")
    sim = pd.read_csv(os.path.join(datadir, 'RE400_LDC_GROUND_TRUTH_51X51.csv'))
    sim['x'], sim['y'] = sim['x'], sim['y'] - y_u
    X_train = np.vstack([sim.x.values, sim.y.values, vdx * np.ones_like(sim.x.values), vdy * np.ones_like(sim.y.values)]).T
    y_train = sim[['u', 'v', 'p']].values
 
   
print ('# training sample = %3d  (dx = %.2e, dy = %.2e)' %(len(y_train), vdx, vdy))
    

# view
fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(1,2,1); _sc = 1.
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train[:, 0], marker='H', s=10, alpha=.85, cmap='rainbow'); plt.title('u-vel');
ax1 = fig.add_subplot(1,2,2); _sc = .5
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train[:, 1], marker='H', s=10, alpha=.85, cmap='rainbow'); plt.title('v-vel');

# BC sample

bc = np.where((X_train[:, 0] == x_u) | (X_train[:, 0] == x_l) | (X_train[:, 1] == y_u) | (X_train[:, 1] == y_l))[0]
X_bc, y_bc = X_train[bc], y_train[bc]
    
  
print ('# BC sample = %d' %len(y_bc))
print(X_bc.shape, y_bc.shape)
bc_req_len = 4*(n-1)
if (len(y_bc) != bc_req_len ):
    print('x_bc and y_bc wrong. Should be 4x(n-1)')
    exit()

# view
fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(1,2,1); _sc = 1.
plt.scatter(X_bc[:, 0], X_bc[:, 1], c=y_bc[:, 0], marker='H', s=10, alpha=.85, cmap='rainbow'); plt.title('u-vel');
ax1 = fig.add_subplot(1,2,2); _sc = .5
plt.scatter(X_bc[:, 0], X_bc[:, 1], c=y_bc[:, 1], marker='H', s=10, alpha=.85, cmap='rainbow'); plt.title('v-vel');

  

In [ ]:
# PDE
BPE = 32
LR = 1e-3

print(r'Running new Re, $Re=$%d' %Re)

# evaluation sample
X_eval, y_eval = X_train, y_train

# PDE sample
X_pde, y_pde = X_train, y_train

# IC sample
X_ic, y_ic = X_bc, y_bc


# initiate NN model (& pathway to internal values)
n_ffs = 32
lmbda = 1.0

ff = 'SIREN'
sigma = 1.0

# select scheme : 'a-pde', 'n-pde', 'can-pde' 
scheme = 'can-pde'

nn, eval_ins = create_nn(scheme, ff, n_ffs, sigma, lmbda, n_nodes = 20, acf = 'swish', lr_int = 0.001)

# first pass
u_0, v_0, p_0, pde_mse_0, bc_mse_0 = eval_ins(X_eval) 
  
nn.summary()

# PINN training setting: EPOCH, learning rate & sampling strategy

DGEN = SamplingPlan_Fix(data=(X_pde, y_pde, X_ic, y_ic, X_bc, y_bc), batch_size=(Nsamples,0,Nbatch), batch_per_epoch=BPE)

# callback setting: learning rate schedule & training history
lr_sched = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=5e-6)
tr_hist = TrainingHistory(eval_ins, X_eval)
# callbacks_list = [lr_sched, tr_hist]

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose = 0)
callbacks_list = [lr_sched, tr_hist, cp_callback]
    
K.set_value(nn.optimizer.lr, LR)  # set learning rate

# time it
t0 = time()
if(Restart_sol == False ):
    
    # train NN model
    history = nn.fit(DGEN, epochs=EPOCH, verbose=2, callbacks=[callbacks_list])
  
else:
    # train from saved chkpoint model
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    nn.load_weights(latest)
    K.set_value(nn.optimizer.lr, LR) 
    history = nn.fit(DGEN, epochs=EPOCH, verbose=2, callbacks=[callbacks_list])
    
print ("...\nRunning time: %d mins %d secs!" %(int(time()-t0)/60, np.remainder(int(time()-t0), 60)))

In [ ]:
    # final loss
    name_checkpt = 'EPOCH = %05d  LOSS = %.2e  (DATA_U = %.2e, DATA_V = %.2e, BC = %.2e, PDE = %.2e)'\
                    %(EPOCH, history.history['loss'][-1], history.history['u_loss'][-1], history.history['v_loss'][-1],
                      history.history['bc_loss'][-1], history.history['pde_loss'][-1])
    print (name_checkpt)

    # plot loss history
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig, axes = plt.subplots(1, figsize=(12, 5))
    plt.plot(hist['epoch'], hist['bc_loss'], label=r'BC $residual$', alpha=1, c='orange');
    plt.plot(hist['epoch'], hist['pde_loss'], label=r'$%s$-PDEs $residual$'%(scheme.split('-')[0]), alpha=1, c='salmon');
    plt.plot(hist['epoch'], hist['loss'], label=r'Training $loss$', alpha=1, c='royalblue'); plt.yscale('log'); plt.grid();
    plt.plot(hist['epoch'], hist['u_loss'], label=r'u $loss$', alpha=1, c='springgreen');
    plt.plot(hist['epoch'], hist['v_loss'], label=r'v $loss$', alpha=1, c='green');
    plt.plot(hist['epoch'], hist['lr'], "k--", label=r'Learning rate', alpha=.8, linewidth=1);
    plt.xlabel('Epoch', size='x-large'); plt.ylabel('Loss', size='x-large'); plt.xlim((0, EPOCH)); plt.ylim((1e-7, 1e1));
    plt.title(r'Training history', fontsize="x-large"); plt.legend(fontsize='x-large', ncol=2);



    # new prediction & error on [train] data
    p_u, p_v, p_p, _, _ = eval_ins(X_train)

    # mse
    mse_u =  np.mean((p_u.flatten() - y_train[:, 0].flatten())**2)
    mse_v =  np.mean((p_v.flatten() - y_train[:, 1].flatten())**2)
    mse_p =  np.mean((p_p.flatten() - y_train[:, 2].flatten())**2)
    print ('mse u  = %.3e' %mse_u)
    print ('mse v  = %.3e' %mse_v)
    print ('mse p  = %.3e' %mse_p)

In [ ]:
   # visualize flow prediction
fig = plt.figure(figsize=(20, 10))
x = np.linspace(x_l, x_u, n)
y = np.linspace(y_l, y_u, n)
con_lv = 15
def accur(ex_fv, pr_fv):
  ex_norm = np.linalg.norm(ex_fv)
  pr_norm = np.linalg.norm(pr_fv)
  accu = 100.*(1. - (abs(ex_norm-pr_norm)/ex_norm))  
  return accu
_pu, _pv, _pp = p_u.reshape(n, n), p_v.reshape(n, n), p_p.reshape(n, n)
_u, _v, _p = y_train[:, 0].reshape(n, n), y_train[:, 1].reshape(n, n), y_train[:, 2].reshape(n, n)

psi = np.zeros((n,n))
for i in range(n-1):
    for j in range(n-1):
        psi[i+1,j+1] = psi[i,j+1]+ _pu[i+1, j+1]*(y[j+1]-y[j]) #@- _pv[i+1, j+1]*vdx
        
# prediction
ax1 = fig.add_subplot(2,3,1)
plt.contourf(_pu, con_lv, origin='lower', cmap='rainbow', extent=ext, aspect='auto');
plt.colorbar(); plt.xlabel('x'); plt.ylabel('y'); plt.title(r'[U] prediction, $Re=$%d' %Re, size='x-large');
ax1 = fig.add_subplot(2,3,2)
plt.contourf(_pv, con_lv, origin='lower', cmap='rainbow', extent=ext, aspect='auto');
plt.colorbar(); plt.xlabel('x'); plt.ylabel('y'); plt.title(r'[V] prediction, $Re=$%d' %Re, size='x-large');
ax1 = fig.add_subplot(2,3,3)
plt.contourf(_pp, con_lv, origin='lower', cmap='rainbow', extent=ext, aspect='auto');
plt.colorbar(); plt.xlabel('x'); plt.ylabel('y'); plt.title(r'[P] prediction, $Re=$%d' %Re, size='x-large');
ax1 = fig.add_subplot(2,3,4)
plt.contourf(psi, con_lv, origin='lower', cmap='BrBG', extent=ext, aspect='auto');
plt.colorbar(); plt.xlabel('x'); plt.ylabel('y'); plt.title(r'psi prediction, $Re=$%d' %Re, size='x-large');
# simulation
#ax1 = fig.add_subplot(2,3,4)
#plt.contourf(_u, con_lv, origin='lower', cmap='rainbow', extent=ext, aspect='auto');
#plt.colorbar(); plt.xlabel('x'); plt.ylabel('y'); plt.title(r'[U] simulation, $Re=$%d' %Re, size='x-large');
#ax1 = fig.add_subplot(2,3,5)
#plt.contourf(_v, con_lv, origin='lower', cmap='rainbow', extent=ext, aspect='auto');
#plt.colorbar(); plt.xlabel('x'); plt.ylabel('y'); plt.title(r'[V] simulation, $Re=$%d' %Re, size='x-large');

plt.savefig('hdg_results_r1000_1K_second', transparent= True)

y_array = [0, 0.0547, 0.0625, 0.0703, 0.1016, 0.1719, 0.2813,
                   0.4531, 0.50, 0.6172, 0.7344, 0.8516, 0.9531, 
                   0.9609, 0.9688, 0.9766, 1.0]
    
u1000_array = [0.0, -0.1794, -0.2019, -0.2238, -0.3018, -0.3883, -0.2794, -0.1079,
                   -0.0620, 0.0580, 0.1908, 0.3383, 0.4776, 0.5284, 0.6009, 0.6756, 1.0]
    
fig1 = plt.figure(figsize=(12, 5))
ax1 = fig1.add_subplot(2, 3, 1)
nxp = int((n/2)+1)
plt.plot( _pu[:, nxp], x, 'r', label ="Re=1000")
plt.plot( u1000_array, y_array, 'ro')
plt.xlabel('x'); plt.ylabel('y'); plt.title(r'mid section u vel', size='x-large');
legend = ax1.legend(loc='best', shadow=True)
legend.get_frame().set_facecolor('white')
    
x_array = [0.0, 0.0625, 0.0703, 0.0781,0.0938,
                   0.1563, 0.2266, 0.2344, 0.50, 0.8047,
                   0.8594, 0.9063, 0.9453, 0.9531,
                   0.9609, 0.9688, 1.0]
v1000_array = [0.0, 0.2802, 0.2968, 0.3112, 0.3330, 0.3767, 0.3333, 0.3240, 0.0258,
                   -0.3221, -0.4306, -0.5265, -0.3948, -0.3464, -0.2798, -0.2103, 1.0]
ax2 = fig1.add_subplot(2, 2,  2)
nxp = int((n)/2 + 1)
plt.plot( _pv[nxp, :], x, 'm', label ="Re1000")
plt.plot( v1000_array, x_array, 'm.')
plt.xlabel('x'); plt.ylabel('y'); plt.title(r'mid section v vel, $Re=$%d' %Re, size='x-large');
legend = ax2.legend(loc='best', shadow=True)
legend.get_frame().set_facecolor('white')
plt.savefig('hdg_uv_plots_K1_second,Re=%d' %Re, transparent= True)
       

